#####   Getting and preprocessing input data

In [ ]:
# Set working directory and path to shapefile of study area

working_dir = 'volta'  #provide working folder directory
study_area = 'bakaano/test/volta.shp'  #provide path to shapefile of study_area
start_date = '1981-01-01'  # provide start date in format e.g. '2016-12-31'
end_date = '1990-12-31'  #provide end date in format e.g. '2016-12-31'

In [ ]:
from bakaano.tree_cover import TreeCover
vf = TreeCover(working_dir, study_area)
vf.download_tree_cover()
vf.preprocess_tree_cover()
vf.plot_tree_cover()

In [ ]:
from bakaano.ndvi import NDVI
nd = NDVI(working_dir, study_area)
nd.download_ndvi()
nd.preprocess_ndvi()
nd.plot_ndvi()

In [ ]:
# Get elevation data

from bakaano.dem import DEM
dd = DEM(working_dir, study_area, local_data=False, local_data_path=None)
dd.get_dem_data()
dd.plot_dem()

In [ ]:
# Get soil data

from bakaano.soil import Soil
sgd = Soil(working_dir, study_area)
sgd.get_soil_data()
sgd.plot_soil(vmax=50)

In [ ]:
# Get meteo data

from bakaano.meteo import Meteo
cd = Meteo(working_dir, study_area, start_date=start_date, end_date=end_date, local_data=False, data_source='CHELSA', local_prep_path=None, local_tasmax_path=None, 
           local_tasmin_path=None, local_tmean_path=None)
prep, tasmax, tasmin, tmean = cd.get_meteo_data()

#####   Training Bakaano-Hydro model 

In [ ]:
#TRAINING THE NEURAL NETWORK MODEL
# Initialize  instance of Bakaano-Hydro model

train_start_date = ''
train_end_date = ''
grdc_netcdf = 'volta/GRDC-Daily-africa-south-america.nc'   #provide path to netcdf format of observed streamflow obtained from GRDC

from bakaano.runner import BakaanoHydro
bk = BakaanoHydro(  
    working_dir,
    study_area,
    train_start_date,
    train_end_date,
    climate_data_source='CHELSA'
)

bk.train_streamflow_model(grdc_netcdf=grdc_netcdf, prep_nc=prep, tasmax_nc=tasmax, tasmin_nc=tasmin, tmean_nc=tmean, 
                          loss_fn='mean_squared_logarithmic_error', num_input_branch=3, lookback=365, batch_size=256, num_epochs=100)

#####   Evaluating Bakaano-Hydro model performance

In [ ]:
#EVALUATING THE NEURAL NETWORK MODEL
model_path = f'{working_dir}/models/deepstrmm_model_tcn365.keras'
grdc_netcdf = ''   #provide path to netcdf format of observed streamflow obtained from GRDC
val_start_date = ''
val_end_date = ''

from bakaano.runner import BakaanoHydro
bk = BakaanoHydro(
    working_dir,
    study_area,
    val_start_date,
    val_end_date
)

bk.evaluate_streamflow_model(model_path='', grdc_netcdf=grdc_netcdf, prep_nc=prep, tasmax_nc=tasmax, tasmin_nc=tasmin, tmean_nc=tmean,
                             loss_fn='mean_squared_logarithmic_error', num_input_branch=3, lookback=365, batch_size=256)

In [ ]:
#####   Simulating streamflow series with Bakaano-Hydro

In [ ]:
model_path = f'{working_dir}/models/deepstrmm_model_tcn365.keras'
lat = ''
lon = ''

from bakaano.runner import BakaanoHydro
bk = BakaanoHydro(
    working_dir,
    study_area,
    val_start_date,
    val_end_date
)

latlist = []
lonlist =[]
bk.simulate_streamflow_batch(model_path='', latlist=latlist, lonlist=lonlist, grdc_netcdf=grdc_netcdf, prep_nc=prep, tasmax_nc=tasmax, tasmin_nc=tasmin, tmean_nc=tmean,
                             loss_fn='mean_squared_logarithmic_error', num_input_branch=3, lookback=365)